## Testing LLM and Spotify API

In [94]:
##Imports
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from transformers import pipeline
from langchain.llms import HuggingFaceHub

In [95]:
##Functions
def get_song_name(response : str) -> str:
    # Process the response to extract only the song name
    # Assuming the format is like: '"Out of Love" is a song by Avicii.'
    start_quote_index = response.find('"')  # Find the index of the first quote
    end_quote_index = response.find('"', start_quote_index + 1)  # Find the index of the closing quote

    if start_quote_index != -1 and end_quote_index != -1:
        # Extract the song name using slicing
        song_name = response[start_quote_index + 1:end_quote_index]
        return song_name
    else:
        return "Song name not found."
    
def get_track_from_spotify(spotify_client_id, spotify_client_secret, song_name : str) -> str:
    auth_manager = SpotifyClientCredentials(client_id=spotify_client_id, client_secret=spotify_client_secret)
    sp = spotipy.Spotify(auth_manager=auth_manager)
    # Search for the song
    results = sp.search(q=song_name, type='track', limit=1)

    # Extract track information
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        return(f"Found track: {track['name']} by {track['artists'][0]['name']}")
    else:
        return("Song not found.")
    
def prompt_llm(huggingface_api_token, prompt : str, prompt_template : PromptTemplate = None) -> str:
    llm = HuggingFaceHub(
    repo_id="ibm-granite/granite-3.0-3b-a800m-instruct",  # replace with the model name you want
    huggingfacehub_api_token=huggingface_api_token
    )
    if prompt_template:
        llm_chain = LLMChain(llm= llm, prompt= prompt_template)
        response = llm_chain.run(prompt)
    else:
        response = llm(prompt)

    return response

In [96]:

# Replace these with your actual Spotify API credentials
client_id = os.getenv('SPOTIFY_CLIENT_ID')
client_secret = os.getenv('SPOTIFY_CLIENT_SECRET')

huggingface_api_token = os.getenv('HUGGING_FACE_TOKEN')

In [97]:
# Generate recommendation with the LangChain LLM
response = prompt_llm(huggingface_api_token, "Recommend me a song from Metallica. But include in your response only the name of the song, and only one song.")
#print(response)

song_name = get_song_name(response)

In [98]:
print(get_track_from_spotify(client_id, client_secret, song_name))

Found track: Enter Sandman (Remastered) by Metallica


## Using promt template

In [108]:
song_description = 'Upbeat music to dance along'

#Templates
song_template = PromptTemplate(
    input_variables=['description'],
    template='Recommend me a song that matches this description: {song_description}.\
        Include in your response only the name of the song, and only one song.'
)

    
song_name = get_song_name(prompt_llm(huggingface_api_token, song_description, song_template))

In [109]:
print(song_name)
print(get_track_from_spotify(client_id, client_secret, song_name))

Uptown Funk
Found track: Uptown Funk (feat. Bruno Mars) by Mark Ronson
